In [1]:
import numpy as np
import sympy as sp
import string

# ---------------- Helper Functions ----------------

# Euclidean GCD
def euclidean_algo(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# Extended Euclidean GCD
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0
    else:
        g, x1, y1 = extended_gcd(b, a % b)
        x, y = y1, x1 - (a // b) * y1
        return g, x, y

# Multiplicative inverse modulo
def multiplicative_inverse(a, m):
    g, x, _ = extended_gcd(a, m)
    if g != 1:
        return None
    else:
        return x % m

# ---------------- Additive Cipher ----------------
def additive_encrypt():
    plaintext = input("Enter plaintext: ").lower().replace(" ", "")
    key = int(input("Enter key (integer): "))
    ciphertext = ''.join([chr((ord(c)-97 + key) % 26 + 97) for c in plaintext])
    return plaintext, ciphertext, key

def additive_decrypt(key):
    ciphertext = input("Enter ciphertext: ").lower()
    plaintext = ''.join([chr((ord(c)-97 - key) % 26 + 97) for c in ciphertext])
    return ciphertext, plaintext

# ---------------- Multiplicative Cipher ----------------
def multiplicative_encrypt():
    plaintext = input("Enter plaintext: ").lower().replace(" ", "")
    key = int(input("Enter key (integer coprime with 26): "))
    if euclidean_algo(key, 26) != 1:
        print("Key must be coprime with 26")
        return
    ciphertext = ''.join([chr(((ord(c)-97) * key) % 26 + 97) for c in plaintext])
    return plaintext, ciphertext, key

def multiplicative_decrypt(key):
    ciphertext = input("Enter ciphertext: ").lower()
    key_inv = multiplicative_inverse(key, 26)
    plaintext = ''.join([chr(((ord(c)-97) * key_inv) % 26 + 97) for c in ciphertext])
    return ciphertext, plaintext

# ---------------- Affine Cipher ----------------
def affine_encrypt():
    plaintext = input("Enter plaintext: ").lower().replace(" ", "")
    a = int(input("Enter multiplicative key a (coprime with 26): "))
    b = int(input("Enter additive key b: "))
    if euclidean_algo(a, 26) != 1:
        print("Key a must be coprime with 26")
        return
    ciphertext = ''.join([chr(((a*(ord(c)-97) + b) % 26) + 97) for c in plaintext])
    return plaintext, ciphertext, a, b

def affine_decrypt(a, b):
    ciphertext = input("Enter ciphertext: ").lower()
    a_inv = multiplicative_inverse(a, 26)
    plaintext = ''.join([chr(((a_inv*(ord(c)-97 - b)) % 26) + 97) for c in ciphertext])
    return ciphertext, plaintext

# ---------------- Playfair Cipher ----------------
def generate_playfair_matrix(key):
    key = ''.join(key.lower().split()).replace('j', 'i')
    matrix_str = ''
    for char in key:
        if char not in matrix_str and char in string.ascii_lowercase:
            matrix_str += char
    for char in string.ascii_lowercase:
        if char not in matrix_str and char != 'j':
            matrix_str += char
    matrix = [list(matrix_str[i:i+5]) for i in range(0, 25, 5)]
    return matrix

def find_position(matrix, char):
    if char == 'j': char = 'i'
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == char:
                return i, j
    return None

def process_text(text):
    text = ''.join(text.lower().split()).replace('j', 'i')
    result = ''
    i = 0
    while i < len(text):
        a = text[i]
        b = ''
        if i+1 < len(text):
            b = text[i+1]
        if a == b:
            result += a + 'x'
            i += 1
        else:
            if b:
                result += a + b
                i += 2
            else:
                result += a + 'x'
                i += 1
    return result

def clean_decrypted_text(text):
    # Remove 'x' inserted between repeated letters
    result = ''
    i = 0
    while i < len(text):
        if i+2 < len(text) and text[i] == text[i+2] and text[i+1] == 'x':
            result += text[i]
            i += 2
        else:
            result += text[i]
            i += 1
    return result

# ---------------- Encryption ----------------
def playfair_encrypt():
    plaintext = input("Enter plaintext: ").lower()
    key = input("Enter key: ").lower()
    
    matrix = generate_playfair_matrix(key)
    text = process_text(plaintext)
    pad_len = (2 - len(text) % 2) % 2  # padding to make length even if needed
    
    ciphertext = ''
    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)
        if r1 == r2:
            ciphertext += matrix[r1][(c1+1)%5] + matrix[r2][(c2+1)%5]
        elif c1 == c2:
            ciphertext += matrix[(r1+1)%5][c1] + matrix[(r2+1)%5][c2]
        else:
            ciphertext += matrix[r1][c2] + matrix[r2][c1]
    
    return plaintext, ciphertext, pad_len, matrix

# ---------------- Decryption ----------------
def playfair_decrypt(pad_len, matrix):
    ciphertext = input("Enter ciphertext: ").lower()
    plaintext = ''
    
    for i in range(0, len(ciphertext), 2):
        a, b = ciphertext[i], ciphertext[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)
        if r1 == r2:
            plaintext += matrix[r1][(c1-1)%5] + matrix[r2][(c2-1)%5]
        elif c1 == c2:
            plaintext += matrix[(r1-1)%5][c1] + matrix[(r2-1)%5][c2]
        else:
            plaintext += matrix[r1][c2] + matrix[r2][c1]
    
    # Remove internal 'x's between repeated letters
    plaintext = clean_decrypted_text(plaintext)
    
    # Remove padding 'x' at the end if any
    if pad_len > 0:
        plaintext = plaintext[:-pad_len]
    
    return ciphertext, plaintext

# ---------------- Hill Cipher ----------------
# encryption: c = p * k (mod 26), k is key square matrix
def hill_cipher_encrypt():
    p = input('Enter the plaintext: ').lower() # plaintext
    k_len = int(input('Enter the order of the key matrix (order of [3, 5] is preferred): '))
    k = []

    for i in range(k_len):
        temp = list(map(int, input().split()))
        k.append(temp)

    k = np.array(k)
    k_det = int(round(np.linalg.det(k)) % 26)

    if(euclidean_algo(k_det,  26) != 1):
        print('Enter valid key matrix. The determinant of matrix must be co-prime with 26')
        return

    p_len = len(p)
    
    p_wo_space = p.replace(' ', '')
    p_wo_space_len = len(p_wo_space)

    # padding 'x' if length of the plaintext is not the multiple of the order of the key matrix
    pad_len = (k_len - (p_wo_space_len % k_len)) % k_len
    p_wo_space += 'x' * pad_len
    p_wo_space_len = len(p_wo_space)
    
    ans, c = [], ''
    for i in range(0, p_wo_space_len, k_len):
        item_temp = list(p_wo_space[i:i+k_len]) # taking strings of length k_len at a time
        item_temp = np.array([ord(item) - ord('a') for item in item_temp]).reshape(-1, 1) # converting it to numpy array so that req. methods can be used
        ans_temp = (np.dot(k, item_temp) % 26).reshape(-1)

        # converting the ascii code to the character and also spaces accordingly
        for item in ans_temp:
            c += chr(ord('a') + int(item))
            
    return p, c, pad_len

# P = K−1 * C (mod 26)
# K−1 = (det K)−1 * adj(K) (mod 26)
def hill_cipher_decrypt(pad_len):
    c = input('Enter the ciphertext: ').lower() # ciphertext
    k_len = int(input('Enter the order of the key matrix (order of [3, 5] is preferred): '))
    k = []

    for i in range(k_len):
        temp = list(map(int, input().split()))
        k.append(temp)

    k = sp.Matrix(k)
    
    k_adj = k.adjugate() # finding the adjugate of k
    k_det_inverse = multiplicative_inverse(int(k.det()) % 26, 26) # finding the inverse of determinant of k
    
    k_inverse = np.array((k_det_inverse * k_adj) % 26).astype(np.int32)

    p = ''
    c_len = len(c)

    for i in range(0, c_len, k_len):
        block = list(c[i:i+k_len:])
        block = np.array([ord(item) - ord('a') for item in block]).reshape(-1, 1)
        ans_block = (np.mod(np.dot(k_inverse, block), 26)).reshape(-1)

        for item in ans_block:
            p += chr(ord('a') + int(item))

    if(pad_len > 0):
        p = p[:-pad_len:]
    
    return c, p

In [3]:
# ---------------- Main Section ----------------
def main():
    print('-----GCD by Euclidean Algorithm-----')
    ans = euclidean_algo(2, 3)
    print('gcd({}, {}) = {}'.format(2, 3, ans))

    print('-----GCD by extended Euclidean Algorithm------')
    ans = extended_gcd(2, 3)
    print('gcd({}, {}) = {} and x = {}, y = {}'.format(2, 3, ans[0], ans[1], ans[2]))

    print('-----Modulo Inverse-----')
    ans = multiplicative_inverse(7, 26)
    print('Inverse of 7 = {}'.format(ans))
    
    print("----- Additive Cipher -----")
    plaintext, ciphertext, key = additive_encrypt()
    print(f"Plaintext -> {plaintext} \t Ciphertext -> {ciphertext}")
    c_text, p_text = additive_decrypt(key)
    print(f"Ciphertext -> {c_text} \t Plaintext -> {p_text}\n")

    print("----- Multiplicative Cipher -----")
    plaintext, ciphertext, key = multiplicative_encrypt()
    print(f"Plaintext -> {plaintext} \t Ciphertext -> {ciphertext}")
    c_text, p_text = multiplicative_decrypt(key)
    print(f"Ciphertext -> {c_text} \t Plaintext -> {p_text}\n")

    print("----- Affine Cipher -----")
    plaintext, ciphertext, a, b = affine_encrypt()
    print(f"Plaintext -> {plaintext} \t Ciphertext -> {ciphertext}")
    c_text, p_text = affine_decrypt(a, b)
    print(f"Ciphertext -> {c_text} \t Plaintext -> {p_text}\n")

    print("----- Playfair Cipher -----")
    plaintext, ciphertext, pad_len, matrix = playfair_encrypt()
    print(f"Plaintext -> {plaintext} \t Ciphertext -> {ciphertext}")
    c_text, p_text = playfair_decrypt(pad_len, matrix)
    print(f"Ciphertext -> {c_text} \t Plaintext -> {p_text}\n")

    print("----- Hill Cipher -----")
    encrypt_code = hill_cipher_encrypt()
    print('Plaintext -> {} \t Ciphertext -> {}'. format(encrypt_code[0], encrypt_code[1]))
    if(encrypt_code[2] != None):
        decrypt_code = hill_cipher_decrypt(encrypt_code[2])
    print('Ciphertext -> {} \t Plaintext -> {}'. format(decrypt_code[0], decrypt_code[1]))

# Run the main section
if __name__ == "__main__":
    main()


-----GCD by Euclidean Algorithm-----
gcd(2, 3) = 1
-----GCD by extended Euclidean Algorithm------
gcd(2, 3) = 1 and x = -1, y = 1
-----Modulo Inverse-----
Inverse of 7 = 15
----- Additive Cipher -----


Enter plaintext:  hello
Enter key (integer):  3


Plaintext -> hello 	 Ciphertext -> khoor


Enter ciphertext:  khoor


Ciphertext -> khoor 	 Plaintext -> hello

----- Multiplicative Cipher -----


...

Ciphertext -> gyizscokcfbu 	 Plaintext -> helloworldx

----- Hill Cipher -----


Enter the plaintext:  pay more money
Enter the order of the key matrix (order of [3, 5] is preferred):  3
 17 17 5
 21 18 21
 2 2 19


Plaintext -> pay more money 	 Ciphertext -> lnshdlewmtrw


Enter the ciphertext:  lnshdlewmtrw
Enter the order of the key matrix (order of [3, 5] is preferred):  3
 17 17 5
 21 18 21
 2 2 19


Ciphertext -> lnshdlewmtrw 	 Plaintext -> paymoremoney
